In [ ]:
# Imports

import car
from auto.capabilities import acquire, release, list_caps
from car.motors import set_throttle, set_steering
from auto import console
import time


import numpy as np
import cv2
import os

In [ ]:
# Computer Vision constants
cv2.setNumThreads(0)

blue = np.array([112,121, 189])
green = np.array([100, 110, 100])
diff = 30

lowerb = blue - diff
upperb = blue + diff
lowerb = np.clip(lowerb, 0, 255)
upperb = np.clip(upperb, 0, 255)

lowerg = green - diff 
upperg = green + diff
lowerg = np.clip(lowerg, 0, 255)
upperg = np.clip(upperg, 0, 255)

crop_height = 70
crop_left = 60
crop_right = 60

In [ ]:
# Functions

def crop(img):
    """
    Camera captures 240 pixels of height and 320 of width
    Shape is (240, 320, 3)
    """
    return img[-crop_height:, crop_left:-crop_right]


def bottom_half(img):
    """
    Cut out the top half
    """
    return img[-120:, :]


def scale(img, scale_percent):
    # Calculate the new dimensions
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    new_size = (width, height)

    # Resize the image:
    img = cv2.resize(img, newSize, None, None, None, cv2.INTER_AREA)
    return img


def percent_blue(img, scale_percent=1):
    # Convert the image to HSV:
    #hsvImage = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    rgb_image = img
    #cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Create the HSV mask
    #hsvMask = cv2.inRange(hsvImage, lower, upper)
    
    rgb_mask = cv2.inRange(rgb_image, lowerb, upperb)
    # AND mask & input image:
    # rgbOutput = cv2.bitwise_and(img, img, mask=rgbMask)
    
    # You can use the mask to count the number of white pixels.
    # Remember that the white pixels in the mask are those that
    # fall in your defined range, that is, every white pixel corresponds
    # to a blue pixel. Divide by the image size and you got the
    # percentage of blue pixels in the original image:
    ratio_blue = cv2.countNonZero(rgb_mask)/(img.size/3)

    # This is the color percent calculation, considering the resize I did earlier.
    color_percent = (ratio_blue * 100) / scale_percent 

    return color_percent


def percent_green(img, scale_percent=1):
    rgb_image = img
    rgb_mask = cv2.inRange(rgb_image, lowerg, upperg)
    ratio_green = cv2.countNonZero(rgb_mask)/(img.size/3)
    color_percent = (ratio_green * 100) / scale_percent 
    return color_percent

In [ ]:
# Racing variables

turns = [70]
NUM_LAPS = 30
MAX_THROTTLE = 25
MIN_THROTTLE = 18

In [ ]:
# Custom logic

gyroscope = acquire('Gyroscope_accum')
gyroscope.read()

on_straight = True
turn_number = 0 # Index into turns array
lap = 0

steer_bias = -6.0 # The measured offset that the car is from straight (this is a motor problem, the gyroscope understands that it's 6 degrees off)

blue_threshold = 1.0 # As a percent
angle_threshold = 15.0 # If greater than this on a straight and approaching a boundary reduce the throttle

def start():
    return -5, maxThrottle


def loop(prev_steer, prev_throttle):
    steer = 0
    steer_direction = 1
    
    throttle = prev_throttle

    if on_straight:
        z = gyroscope.read()[2]
        if z > 0:
            steer_direction = -1

        steer = np.abs(z)/2  
        
        frame = car.capture()
        blue_img = crop(frame)
        # green_img = bottom_half(frame)
        blue_percent = percent_blue(blue_img)
        if percent > blue_threshold and steer > angle_threshold:
            throttle -= 20
        else:
            throttle += 10
                
    else:
        pass
    
    
    throttle = min(max(min_throttle, throttle), max_throttle) # Clip it on either side
    
    return (steer * steer_direction) + steer_bias, throttle

In [ ]:
f = car.capture()

steer, throttle = start()

set_throttle(throttle)
set_steering(steer)

time.sleep(0.5)


# while lap < 10:
#     if turnNumber != 0 and turnNumber % len(turns) == 0:
#         lap += 1

#     d_steer, d_throttle = loop()
#     steer += d_steer
#     throttle += d_throttle
#     set_steering(steer)
#     set_throttle(throttle)
#     time.sleep(0.1) # Each command runs for 1 second
    
# release('Gyroscope_accum')

while lap < numLaps:
    steer, throttle = loop(steer, throttle)
    set_throttle(throttle)
    set_steering(steer)
    time.sleep(0.1)
    lap += 1
    console.print(gyroscope.read()[2])
    
release(gyroscope)

In [ ]:
f = car.capture()
car.plot(f)